In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 7, 31)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,16,S,BCH,3000,21,0,CP1S,SET100,2022-08-31


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,BCH,1.0000,0.0000,0.2000,0.0000,1.2000,"21,000",2022-08-31,2022-09-15,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'BCH'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,610,BCH,26.75,16.60,I,0.00,21.00,507.61,-0.41,20.90,"3,000",-8,10,"21,000",1,0,CP1S,SET100


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

16

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AP,10000,9.50,0,MinP,SET100
1,O,ASK,3000,30.00,0,52WL,SET
2,O,BANPU,12000,11.50,0,1L,SET50
3,O,BCPG,9000,10.00,0,52WL,SET100
4,O,DELTA,300,450.00,0,52WL,SET
5,O,IVL,3000,43.50,0,13WL,SET50
6,B,MCS,5000,10.60,0,52WL,SET
7,O,RBF,9000,11.40,0,52WL,SET100
8,B,RCL,3000,35.25,0,RD20pct,SET100
9,O,SINGER,3000,35.50,0,52WL,SET100


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,DIF,10000,14.6,0,CUT,SET
1,I,BCH,3000,21.0,0,CP1S,SET100


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(16, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,3,3,3,3,3,3
I,2,2,2,2,2,2
O,11,11,11,11,11,11


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,3,3,3,3,3,3
O,11,11,11,11,11,11
S,2,2,2,2,2,2


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(83, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(89, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
3,O,BCPG,9000.0,10.00,0.0,52WL,SET100,NaT,left_only
4,O,DELTA,300.0,450.00,0.0,52WL,SET,NaT,left_only
7,O,RBF,9000.0,11.40,0.0,52WL,SET100,NaT,left_only
9,O,SINGER,3000.0,35.50,0.0,52WL,SET100,NaT,left_only
11,O,SSP,10000.0,8.75,0.0,52WL,SET,NaT,left_only
13,O,TTB,90000.0,0.95,0.0,52WL,SET50,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,3,3,3,3,3,3,3
O,11,11,11,11,11,11,11
S,2,2,2,2,2,2,2


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,O,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,O,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,O,BCPG,9000,10.00,0,52WL,SET100,NaT
4,O,DELTA,300,450.00,0,52WL,SET,NaT
5,O,IVL,3000,43.50,0,13WL,SET50,2022-05-31
7,O,RBF,9000,11.40,0,52WL,SET100,NaT
9,O,SINGER,3000,35.50,0,52WL,SET100,NaT
10,O,SNC,9000,16.50,0,1L,SET,2022-03-02
11,O,SSP,10000,8.75,0,52WL,SET,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
6,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
8,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
12,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
15,S,BCH,3000,21.0,0,CP1S,SET100,2022-08-31
14,S,DIF,10000,14.6,0,CUT,SET,2022-08-10


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
6,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
8,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
12,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
0,O,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,O,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,O,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,O,BCPG,9000,10.00,0,52WL,SET100,NaT
4,O,DELTA,300,450.00,0,52WL,SET,NaT
5,O,IVL,3000,43.50,0,13WL,SET50,2022-05-31
7,O,RBF,9000,11.40,0,52WL,SET100,NaT


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,14,14,14,14,14,14,14
S,2,2,2,2,2,2,2


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

16

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'MCS', 5000, 10.6, 0, '52WL', 'SET', '2022-09-03']
['B', 'RCL', 3000, 35.25, 0, 'RD20pct', 'SET100', '2022-08-27']
['B', 'TMT', 12000, 7.9, 0, 'RD20pct', 'SET', '2022-08-27']
['B', 'AP', 10000, 9.5, 0, 'MinP', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 30.0, 0, '52WL', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11']
['B', 'BCPG', 9000, 10.0, 0, '52WL', 'SET100', 'NaT']
['B', 'DELTA', 300, 450.0, 0, '52WL', 'SET', 'NaT']
['B', 'IVL', 3000, 43.5, 0, '13WL', 'SET50', '2022-05-31']
['B', 'RBF', 9000, 11.4, 0, '52WL', 'SET100', 'NaT']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', 'NaT']
['B', 'SNC', 9000, 16.5, 0, '1L', 'SET', '2022-03-02']
['B', 'SSP', 10000, 8.75, 0, '52WL', 'SET', 'NaT']
['B', 'TTB', 90000, 0.95, 0, '52WL', 'SET50', 'NaT']
['S', 'DIF', 10000, 14.6, 0, 'CUT', 'SET', '2022-08-10']
['S', 'BCH', 3000, 21.0, 0, 'CP1S', 'SET100', '2022-08-31']


In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,NaT
7,8,B,DELTA,300,450.00,0,52WL,SET,NaT
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,NaT


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
7,8,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,2022-02-02


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
1,2,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
2,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
3,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
4,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
5,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
6,7,B,BCPG,9000,10.00,0,52WL,SET100,NaT
7,8,B,DELTA,300,450.00,0,52WL,SET,NaT
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,10,B,RBF,9000,11.40,0,52WL,SET100,NaT


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

6

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [33]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
1,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
2,14,B,TTB,90000,0.95,0,52WL,SET50,2022-02-02


In [34]:
name = 'TOP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.00000,60.000000,60.000000,6.000000e+01,60.000000
mean,54.16250,54.812500,53.408333,1.749782e+07,54.129167
std,3.52107,3.572086,3.491808,1.248356e+07,3.499846
min,49.25000,49.750000,48.250000,3.331129e+06,49.750000
25%,51.00000,51.250000,50.000000,9.321204e+06,50.500000
50%,54.00000,55.000000,53.125000,1.509458e+07,54.250000
75%,57.31250,58.062500,56.750000,2.053557e+07,57.312500
max,60.75000,62.250000,60.500000,6.425642e+07,61.000000


### 2) Set100 records

In [35]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,7,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
2,10,B,RBF,9000,11.40,0,52WL,SET100,2022-02-02
3,2,B,RCL,3000,35.25,0,RD20pct,SET100,2022-08-27
4,11,B,SINGER,3000,35.50,0,52WL,SET100,2022-02-02
5,16,S,BCH,3000,21.00,0,CP1S,SET100,2022-08-31


In [36]:
name = 'KKP'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,66.683333,67.425000,66.029167,5.815864e+06,66.808333
std,4.058854,4.011187,3.953408,3.606694e+06,3.989225
min,60.500000,61.500000,59.500000,1.763119e+06,60.750000
25%,62.750000,63.500000,62.250000,3.621307e+06,63.000000
50%,66.625000,67.625000,65.625000,4.724210e+06,66.500000
75%,70.500000,71.312500,69.750000,6.964587e+06,70.562500
max,73.000000,73.750000,72.250000,2.203634e+07,73.500000


### 3) Set records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
1,8,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
2,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03
3,12,B,SNC,9000,16.50,0,1L,SET,2022-03-02
4,13,B,SSP,10000,8.75,0,52WL,SET,2022-02-02
5,3,B,TMT,12000,7.90,0,RD20pct,SET,2022-08-27
6,15,S,DIF,10000,14.60,0,CUT,SET,2022-08-10


In [38]:
name = 'AIT'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,120.000000,120.000000,120.000000,1.200000e+02,120.000000
mean,6.562917,6.703750,6.459167,1.272667e+07,6.570000
std,0.617650,0.654659,0.597227,1.981702e+07,0.618259
min,5.550000,5.650000,5.500000,8.393650e+05,5.550000
25%,5.987500,6.087500,5.900000,2.937804e+06,6.000000
50%,6.700000,6.850000,6.600000,6.195226e+06,6.700000
75%,7.000000,7.150000,6.900000,1.373169e+07,7.000000
max,8.050000,8.350000,7.900000,1.238636e+08,8.050000


### After call Ord-log.bat

In [39]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [40]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [41]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,AP,-5,MinP,10000,9.50,9.75,+0.52%,0,2022-05-10
1,Buy,ASK,-24,52WL,3000,30.00,36.00,+0.70%,0,2022-03-03
2,Buy,BANPU,-19,1L,12000,11.50,13.40,+2.29%,0,2022-04-11
3,Buy,BCPG,-2,52WL,9000,10.00,10.20,-0.97%,0,2022-02-02
4,Buy,DELTA,-150,52WL,300,287.00,474.00,+28.80%,0,2022-02-02
5,Buy,IVL,-1,13WL,3000,43.50,43.75,-0.57%,0,2022-05-31
6,Buy,MCS,-9,52WL,5000,10.60,11.50,0.00%,0,2022-09-03
7,Buy,RBF,7,52WL,9000,13.50,12.80,+5.79%,0,2022-02-02
8,Buy,RCL,-9,RD20pct,3000,35.25,37.50,+2.04%,0,2022-08-27
9,Buy,SINGER,-24,52WL,3000,35.50,41.50,+1.22%,0,2022-02-02
